In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import xgboost as xgb # brew install xgboost

In [52]:
target_columns = ['isAnger', 'isDisgust', 'isFear', 'isHappiness', 'isSadness', 'isSurprise', 'emotion']

In [53]:
df = pd.read_csv('train.csv', index_col=0)
df = df.set_index('video')

In [54]:
df = df.fillna(0)

In [55]:
X = df.drop(target_columns, axis=1)
X = X.astype(float)
X = (X + 300) / 600
X

,0-1_left_eyebrow0X,0-1_left_eyebrow0Y,0-1_left_eyebrow1X,0-1_left_eyebrow1Y,0-1_right_eyebrow0X,0-1_right_eyebrow0Y,0-1_right_eyebrow1X,0-1_right_eyebrow1Y,0-1_nose_tip0X,0-1_nose_tip0Y,...,30-31_top_lip1X,30-31_top_lip1Y,30-31_bottom_lip0X,30-31_bottom_lip0Y,30-31_bottom_lip1X,30-31_bottom_lip1Y,0-1_chin0X,0-1_chin0Y,0-1_nose_bridge0X,0-1_nose_bridge0Y
video,,,,,,,,,,,,,,,,,,,,,
trainAnger001,0.500000,0.500000,0.498333,0.500000,0.500000,0.501667,0.500000,0.500000,0.500000,0.5,...,0.500000,0.498333,0.500000,0.500000,0.500000,0.500000,0.5,0.5,0.5,0.5
trainAnger002,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.501667,0.501667,0.500000,0.5,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.5,0.5,0.5,0.5
trainAnger003,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.501667,0.501667,0.500000,0.5,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.5,0.5,0.5,0.5
trainAnger004,0.501667,0.500000,0.500000,0.500000,0.500000,0.501667,0.498333,0.501667,0.500000,0.5,...,0.500000,0.501667,0.501667,0.501667,0.500000,0.501667,0.5,0.5,0.5,0.5
trainAnger005,0.498333,0.500000,0.500000,0.500000,0.498333,0.498333,0.500000,0.498333,0.500000,0.5,...,0.500000,0.501667,0.500000,0.500000,0.501667,0.500000,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
trainSurprise067,0.500000,0.498333,0.500000,0.500000,0.498333,0.501667,0.500000,0.498333,0.498333,0.5,...,0.500000,0.501667,0.500000,0.500000,0.500000,0.500000,0.5,0.5,0.5,0.5
trainSurprise068,0.500000,0.500000,0.498333,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.5,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.498333,0.5,0.5,0.5,0.5
trainSurprise069,0.500000,0.501667,0.500000,0.500000,0.501667,0.501667,0.500000,0.501667,0.500000,0.5,...,0.500000,0.500000,0.500000,0.501667,0.500000,0.501667,0.5,0.5,0.5,0.5


In [56]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from numpy import mean
from numpy import std

X_train, X_test, y_train, y_test = train_test_split(X, df['emotion'], test_size=0.2)


clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

In [57]:
prediction = grid.predict(X_test)

In [58]:
success = 0
for row, pred in zip(y_test.iteritems(), prediction):
    if row[1] == pred:
        success += 1
print(f"Accuracy for XGBClassifier(): {success/len(prediction)}")

Accuracy for XGBClassifier(): 0.17647058823529413


# the lines below are obsolete

In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std

X_train, X_test, y_train, y_test = train_test_split(X, df['emotion'], test_size=0.2)
# print(X_train.shape)
# print(y_train.shape)
# gbr = xgb.XGBRegressor(objective='reg:squarederror',
#        colsample_bytree = 0.3,
#        learning_rate=0.01, max_depth=5, alpha=1, 
#        n_estimators=10000)
# gbr.fit(X_train, y_train)
# print("\tModel Accuracy: %.3f" % gbr.score(X_test, y_test))

# model = XGBClassifier(use_label_encoder=False)
# model.fit(X_train, y_train)
# cv = RepeatedStratifiedKFold(n_splits=12, n_repeats=3)
# n_scores = cross_val_score(model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)
# # report performance
# print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

def train_models(X_train, y_train):
#     models = {}
#     for target in target_columns:
    model = XGBClassifier(
#         use_label_encoder=False, objective='objective=multi:softmax', 
#                           max_depth=7, n_estimators=1000, num_classes=6
    )
    model.fit(X_train, y_train)
#     models[target] = model
    return model

def predict_with_models(X_test, models):
    result = pd.DataFrame(index=X_test.index)
    for name in models:
        prediction = pd.DataFrame(models[name].predict(X_test), columns=['pred_' + name], index=X_test.index)
        result = result.join(prediction)
    return result

models = train_models(X_train, y_train)
result = predict_with_models(X_test, models)
result.describe()

TypeError: 'XGBClassifier' object is not iterable

In [130]:
def set_emotion_num(df):
    df.loc[(df.isAnger == 1), 'emotion_num'] = 1
    df.loc[(df.isDisgust == 1), 'emotion_num'] = 2
    df.loc[(df.isFear == 1), 'emotion_num'] = 3
    df.loc[(df.isHappiness == 1), 'emotion_num'] = 4
    df.loc[(df.isSadness == 1), 'emotion_num'] = 5
    df.loc[(df.isSurprise == 1), 'emotion_num'] = 6
    return df

def predict_for_column(df, column_name):
    print("Predicting target column: " + column_name)
    target = df[column_name]
    df = df.drop(target_columns, axis=1)#, errors='ignore')
    
    X_train, X_test, y_train, y_test = train_test_split(df, target, random_state=12, test_size=0.2)

    # Hyperparameters for GradientBoostingRegressor 
#     gbr_params = {'n_estimators': 1000,
#               'max_depth': 3,
#               'min_samples_split': 3,
#               'learning_rate': 0.03,
#               'loss': 'ls'}
#     gbr = GradientBoostingRegressor(**gbr_params)
#     gbr = xgb.XGBRegressor(objective='reg:squarederror',
#                            colsample_bytree = 0.3,
#                            learning_rate=0.01, max_depth=5, alpha=1, 
#                            n_estimators=1000)

    gbr = XGBClassifier(use_label_encoder=False)
#     cv = RepeatedStratifiedKFold(n_splits=20, n_repeats=3)
#     n_scores = cross_val_score(gbr, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
#     print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
#     # Fit the model
    gbr.fit(X_train, y_train)
    print("\tModel Accuracy: %.3f" % gbr.score(X_test, y_test))
#     mse = mean_squared_error(y_test, gbr.predict(X_test))
#     print("\tThe mean squared error (MSE) on test set: {:.4f}".format(mse))
    
    prediction = pd.DataFrame(gbr.predict(X_test), columns=['pred_' + column_name])
    result = y_test.to_frame().reset_index().join(prediction).set_index('video')
    return result, gbr

In [131]:
# df = set_emotion_num(df)
angerResult, angerModel = predict_for_column(df, 'isAnger')
disgustResult, disgustModel = predict_for_column(df, 'isDisgust')
fearResult, fearModel = predict_for_column(df, 'isFear')
happinessResult, happinessModel = predict_for_column(df, 'isHappiness')
sadnessResult, sadnessModel = predict_for_column(df, 'isSadness')
surpriseResult, surpriseModel = predict_for_column(df, 'isSurprise')


Predicting target column: isAnger


XGBoostError: [00:38:51] /Users/travis/build/dmlc/xgboost/src/metric/metric.cc:49: Unknown metric function binary:logistic
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001431fe160 dmlc::LogMessageFatal::~LogMessageFatal() + 112
  [bt] (1) 2   libxgboost.dylib                    0x00000001432d5311 xgboost::Metric::Create(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, xgboost::GenericParameter const*) + 145
  [bt] (2) 3   libxgboost.dylib                    0x00000001432b5408 xgboost::LearnerConfiguration::ConfigureMetrics(std::__1::vector<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > >, std::__1::allocator<std::__1::pair<std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> >, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::al

In [127]:
models = {}
models['anger'] = angerModel
models['disgust'] = disgustModel
models['fear'] = fearModel
models['happiness'] = happinessModel
models['sadness'] = sadnessModel
models['surprise'] = surpriseModel 

In [128]:
results = angerResult.join(disgustResult).join(fearResult).join(happinessResult).join(sadnessResult).join(surpriseResult)
results = results.drop(target_columns, axis=1, errors='ignore')

results['max_value'] = results.max(axis=1)
results['final_prediction'] = results.idxmax(axis=1).str.lstrip('pred_is')
results.to_csv('pampam.csv')

In [129]:
success = 0
for i, row in results.iterrows():
    if row['final_prediction'] in i:
        success += 1

print("Final accuracy: " + str(success/len(results.index)))

Final accuracy: 0.32941176470588235


In [15]:
def do_the_prediction(models, test_df):
#     sc = StandardScaler()
#     test_df = sc.transform(test_df)
    result = pd.DataFrame(test_df.index, columns=['video'])
    
    for key, value in models.items():
        pred = pd.DataFrame(value.predict(test_df), columns=[key])
        result = result.join(pred)
    
    result = result.set_index('video')
    result['label'] = result.idxmax(axis=1)
    result['label'].to_csv('prediction.csv')

test = pd.read_csv('test.csv', index_col=0).set_index('video')
do_the_prediction(models, test)

In [243]:
# 12 kép/videó, GBR
# Final accuracy: 0.5294117647058824, 0.5529411764705883 (random_state=12-re volt allitva)
# 0.5529411764705883 <- ezzel az eredménnyel küldtem el az első predikciót
# 10 képvideó
# Final accuracy: 0.4117647058823529
# 15 kép/videó
# Final accuracy: 0.38823529411764707
# 11 kép/videó:
# Final accuracy: 0.5058823529411764, 0.5176470588235295
# 13 kép/videó:
# Final accuracy: 0.47058823529411764


In [39]:
X_train, X_test, y_train, y_test = train_test_split(df, target, random_state=26, test_size=0.2)

In [40]:
#
# Standardize the dataset
#
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [41]:
#
# Hyperparameters for GradientBoostingRegressor 
#
gbr_params = {'n_estimators': 1000,
          'max_depth': 3,
          'min_samples_split': 3,
          'learning_rate': 0.03,
          'loss': 'ls'}
#
# Create an instance of gradient boosting regressor 
#
gbr = GradientBoostingRegressor(**gbr_params)
#
# Fit the model
#
gbr.fit(X_train_std, y_train)

GradientBoostingRegressor(learning_rate=0.03, min_samples_split=3,
                          n_estimators=1000)

In [45]:
#
# Print Coefficient of determination R^2
#
print("Model Accuracy: %.3f" % gbr.score(X_test_std, y_test))
#
# Create the mean squared error
#
mse = mean_squared_error(y_test, gbr.predict(X_test_std))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

Model Accuracy: 0.304
The mean squared error (MSE) on test set: 0.0958


In [43]:
# GBR 25% a validáló adat, 0.03-es learning rate, 5 kép/videora:
# Model Accuracy: 0.335
# The mean squared error (MSE) on test set: 1.6932
# GBR 25% a validáló adat, 0.03-es learning rate, 10 kép/videora:
# Model Accuracy: 0.354
# The mean squared error (MSE) on test set: 1.8729

In [60]:
preds = pd.DataFrame(gbr.predict(X_test_std), columns=['pred_isAnger'])

In [56]:
y_out = y_test.to_frame().reset_index().join(preds)

In [61]:
y_out['pred_isAnger'] = y_out['pred_isAnger'].round()

KeyError: 'pred_isAnger'

In [58]:
y_out.to_csv('pampam.csv')

In [54]:
#Random forest
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
print("Model Accuracy: %.3f" % clf.score(X_test_std, y_test))
mse = mean_squared_error(y_test, clf.predict(X_test_std))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

Model Accuracy: 0.835
The mean squared error (MSE) on test set: 0.1647
